In [ ]:
import os
import gc
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, LinearSegmentedColormap, colorConverter
from matplotlib import rc
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.transforms import Bbox
# from matplotlib.patches import FancyBboxPatch
from matplotlib.font_manager import FontProperties

import matplotlib as mpl
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm

from functions.mandyocIO import read_mandyoc_output, read_datasets, read_particle_path

# Initial setup

In [ ]:
# Path to the model directory
model_path = "/home/kugelblitz/runs/strak_71"
# model_path = "/home/kugelblitz/runs/_strain-softening-0-a"

read_ascii = True # if False, it will skip the ascii reading and saving processes and it will read the datasets from previous runs

plot_first = False # Generate and plot the first image
plot_last = False # Generate and plot the last image

plot_step = False
step_num = 180

plot_sample = True # Generate and plot key images across the simulation
sample_size = 5 # Number of key images

save_images = False # Generate and save all the possible images
follow_particle = False # Stores x and z position of a particle through the simulation
make_video = False # Make video witll all the possible imagesTrue
make_gif = False # Make an animated gif with the generated video
plot_temperature_anomaly = False # Plot the 'horizontal temperature anomaly' instead of 'temperature'

# Datasets handeling
## Model path

In [ ]:
# Create the output directory to save the dataset
output_path = os.path.join(model_path, "_output")
if not os.path.isdir(output_path):
    os.makedirs(output_path)

model_name = os.path.split(model_path)[1]

# datasets = ("velocity")
datasets = ("temperature", "density", "strain_rate", "velocity")
# datasets = ("temperature", "viscosity", "strain_rate", "surface", "velocity", "density")

### Read ascii outputs and save them as xarray.Datasets

In [ ]:
# Read data and convert them to xarray.Dataset
if (read_ascii):
    ds_data = read_mandyoc_output(
        model_path,
        datasets=datasets,
        parameters_file="param.txt",
        skip=2,
        )

In [ ]:
if (follow_particle):    
    particle_x, particle_z, particle_ID = read_particle_path(
        model_path,
        (2_400.0e3, -100.0e3),
        unit_number=3,
        ncores=np.nan
    )

In [ ]:
if (follow_particle):
    x_disp = abs(particle_x[-1] - particle_x[0])
    z_disp = abs(particle_z[-1] - particle_z[0])
    print("X displacement:", x_disp/1.0e3, "[km]")
    print("Z displacement:", z_disp/1.0e3, "[km]")
    plt.figure(figsize=(x_disp*2/z_disp, 4), facecolor="white")
    plt.title(f'Particle {int(particle_ID)} trajectory')
    plt.plot(particle_x/1.0e3, particle_z/1.0e3, color='grey')
    plt.scatter(particle_x/1.0e3, particle_z/1.0e3, color='grey')
    plt.scatter(particle_x[0]/1.0e3, particle_z[0]/1.0e3, color='blue', label='Begin', zorder=10) # zorder must be big to be on top of the other plots
    plt.scatter(particle_x[-1]/1.0e3, particle_z[-1]/1.0e3, color='red', label='End', zorder=10)
    plt.legend()
    plt.xlabel("x [km]")
    plt.ylabel("z [km]")
    plt.savefig(f"{output_path}/{model_name}_output_trajectory_{int(particle_ID)}.png", dpi=300)

## Read and merge saved datasets into a single dataset

In [ ]:
to_mm_yr = 365 * 24 * 60 * 60 * 1000.0
max_velocity = 100_000_000.0 # [mm/yr]

if (save_images) or (plot_first) or (plot_last) or (plot_step) or (plot_sample):
    dataset = read_datasets(model_path, datasets)

    # Normalize velocity values
    if ("velocity_x" and "velocity_z") in dataset.data_vars:
        v_max = np.max((dataset.velocity_x**2 + dataset.velocity_z**2)**(0.5))
        print(f"ini v_max: {float(v_max):.2e} [m/s]")
        # if (v_max > max_velocity / to_mm_yr):
        #     v_max = max_velocity / to_mm_yr
        #     print(f"new v_max: {v_max:.2e} [m/s]")
        #     # dataset.velocity_x.where(dataset.velocity_x < v_max, 1.0e100)
        #     dataset.velocity_x.values[dataset.velocity_x.values > v_max] = v_max
        #     dataset.velocity_x.values[dataset.velocity_x.values < -v_max] = -v_max
        #     dataset.velocity_z.values[dataset.velocity_z.values > v_max] = v_max
        #     dataset.velocity_z.values[dataset.velocity_z.values < -v_max] = -v_max
        #     v_max = v_max**2
        dataset.velocity_x[:] = dataset.velocity_x[:] / v_max
        dataset.velocity_z[:] = dataset.velocity_z[:] / v_max

    print(dataset)

if (plot_temperature_anomaly):
    dataset['temperature'] = (dataset['temperature'] - dataset['temperature'].mean(dim='x')) / float(dataset.x[1])
# time = 0
# plt.figure(figsize=(20,4))
# plt.imshow(dataset.velocity_x.isel(time=time).T[::-1])
# plt.colorbar()
# plt.figure(figsize=(20,4))
# plt.imshow(dataset.velocity_z.isel(time=time).T[::-1])
# plt.colorbar()

# Output images cofig

In [ ]:
colorbar_text_pos_left = 1.05
colorbar_width = "5%"
colorbar_text_labelpad = -90
air_thickness = 44.0 # in km

### Steps configuration

In [ ]:
start = 0
end = dataset.time.size
step = 1

if (plot_first) and (not plot_last) and (not plot_sample):
    print("Plotting ONLY the first step")
    start = 0
    end = 1
    step = 1
    
if (not plot_first) and (plot_last) and (not plot_sample):
    print("Plotting ONLY the last step")
    start = dataset.time.size - 1
    end = dataset.time.size
    step = 1 

if (plot_first) and (plot_last) and (not plot_sample):
    print("Plotting the first and last steps")
    start = 0
    end = dataset.time.size 
    step = dataset.time.size - 1

if (plot_step) and (not plot_sample):
    print(f"Plotting ONLY the step {step_num}")
    start = step_num
    end = step_num + 1
    step = 1

times = np.arange(start, end, step)

if (plot_sample):
    print("Plotting sample steps")
    times = np.linspace(start, end-1, sample_size)
    times = times.astype(np.int64)

print("step stamps: ", times)


# Generate and save output images

In [ ]:
if (save_images) or (plot_first) or (plot_last) or (plot_step) or (plot_sample):
    aux = 0
    if ("temperature" or ("velocity_x" and "velocity_x")) in dataset.data_vars:
        aux += 1
        if "temperature" in dataset.data_vars:
            if (plot_temperature_anomaly):
                t_min = float(dataset.temperature.min())
            else:
                t_min = 0.0
            t_max = np.max([float(dataset.temperature.max()), np.abs(t_min)])
            if (plot_temperature_anomaly):
                t_min = - np.max([float(dataset.temperature.max()), np.abs(t_min)])
        if ("velocity_x" and "velocity_x") in dataset.data_vars:
            num_vectors = 20
            desired_mm_per_year_value = 50.0
            print(f"v_max: {(float(v_max)/to_mm_yr):.2f} [mm/yr]")
            v_key = desired_mm_per_year_value / to_mm_yr / v_max
            v_scale = 0.75 # [tip:0.5] If set to <n>, max velocity arrows will have the size of dataset.x.max()/<n>, ie the bigger, the smaller will be the arrow
    if "strain" in dataset.data_vars:
        s_max = dataset.strain.max()
        s_min = np.max([dataset.strain.min(), s_max/1.0E3])
        print("strain: ", float(s_min), float(s_max))
        aux += 1
    if "strain_rate" in dataset.data_vars:
        e_max = dataset.strain_rate.max()
        e_min = np.max([dataset.strain_rate.min(), e_max/1.0E3])
        print("strain_rate: ", float(e_min), float(e_max))
        aux += 1
    if "viscosity" in dataset.data_vars:
        eta_min, eta_max = float(dataset.attrs["viscosity_min"]), float(dataset.attrs["viscosity_max"])
        aux += 1
    if "surface" in dataset.data_vars:
        w_min, w_max = -10, 10 # dataset.surface.min()/1.0e3, dataset.surface.max()/1.0e3
        aux += 1
    if "density" in dataset.data_vars:
        aux += 1

    aux_size_x = np.round(float(abs(dataset.x[-1]/1.0e6)), 1)
    aux_size_z = np.round(float(abs(dataset.z[0]/1.0e6)), 1)
    
    aux_size_x = np.max([aux_size_x, 1])
    aux_size_z = np.max([aux_size_z, 1])
    print(aux_size_x, aux_size_z)
    
    # Create the color map to plot the density
    cr = 255.0
    color_air = (255.0 / cr, 255.0 / cr, 255.0 / cr)
    color_continental_crust = (240.0 / cr, 209.0 / cr, 188.0 / cr)
    color_continental_mantle = (228.0 / cr, 156.0 / cr, 124.0 / cr)
    color_oceanic_crust = (207.0 / cr, 226.0 / cr, 205.0 / cr)
    color_oceanic_mantle = (155.0 / cr, 194.0 / cr, 155.0 / cr)
    color_mantle = (155.0 / cr, 0.0 / cr, 155.0 / cr)
    colors = [
        color_continental_crust, 
        color_continental_mantle, 
        color_oceanic_crust,
    ]
    levels = [
        2750.0, # density will show up with <color_continental_crust>
        3354.0, # density will show up with <color_continental_mantle>
        3378.0, # density will show up with <color_oceanic_crust>
        3400.0  # density will show up with <color_oceanic_mantle>
    ]
    cmap_de = ListedColormap(colors)
    cmap_de.set_under(color_air)
    cmap_de.set_over(color_oceanic_mantle)
    norm = BoundaryNorm(levels, ncolors=cmap_de.N)

    # Create the color map to plot the strain rate
    color_0 = colorConverter.to_rgba('white', alpha = 0)
    color_1 = colorConverter.to_rgba('purple', alpha = 1)
    color_2 = colorConverter.to_rgba('black', alpha = 1)
    cmap_sr = LinearSegmentedColormap.from_list('cmap_sr', [color_0, color_1, color_2], 512)

    SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 16, 16, 16

    plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    hspace = 0.2

    # Corrects an unintended fontsize on the first run for the quiverkey
    quiverkey_font = FontProperties()
    quiverkey_font.set_size(MEDIUM_SIZE)

    # for i in range(start, end, step):
    for i in times:
        subplot_num = 0
        per = np.round(100*(i+1-start)/(end-start), 2)
        print(f'output {i}/{end-1}, {per:.2f}%', end='\r')

        data = dataset.isel(time=i)
        
        if (i==start):
            z_text_pos = float(data.z.mean()) / 1.0E3 + (1.0 * air_thickness)
            x_text_pos = (1.0 + 0.5* (colorbar_text_pos_left - 1.0)) * float(data.x.max()) / 1.0E3
            extent = [data.x.min()/1.0e3, 
                      data.x.max()/1.0e3, 
                      data.z.min()/1.0e3 + air_thickness, 
                      data.z.max()/1.0e3 + air_thickness] 

        fig, axs = plt.subplots(aux, 1, figsize=(3*aux_size_x, aux*4*aux_size_z), sharex=True, facecolor="white")
        fig.subplots_adjust(hspace=hspace)

        if "surface" in dataset.data_vars:
            if (i == 0) or (start != 0): 
                data_0 = dataset.isel(time=0)
                h_air = np.round(np.mean(data_0.surface/1.0e3)) # Estimates air layer thickness based on the first step
            im = axs[subplot_num].hlines(0, data.surface.x[0]/1.0e3, data.surface.x[-1]/1.0e3, linestyle="solid", color="grey")
            im = axs[subplot_num].plot(data.surface.x/1.0e3, data.surface/1.0e3 - h_air)
            axs[subplot_num].set_ylim(-10, 10)
            axs[subplot_num].set_ylim(w_min, w_max)
            axs[subplot_num].set_ylabel("Elevation [km]")
            surface_subplot_num = subplot_num
            imshow_box = np.array(axs[subplot_num].get_position()) # Backup Bbox to be used if no other data_vars is in the xarray.Dataset
            subplot_num += 1

        if "viscosity" in dataset.data_vars:
            im = axs[subplot_num].imshow(data.viscosity.T[::-1], 
                                         extent=extent, 
                                         norm=LogNorm(vmin=eta_min, vmax=eta_max),
                                         cmap="viridis",
                                         interpolation='none')
            cax = inset_axes(axs[subplot_num], 
                             width=colorbar_width, 
                             height="100%", 
                             loc='center left', 
                             bbox_to_anchor=(colorbar_text_pos_left, 0., 0.3, 1), 
                             bbox_transform=axs[subplot_num].transAxes, 
                             borderpad=0)
            axs[subplot_num].set_aspect("equal")
            axs[subplot_num].set_ylabel("Depth [km]")
            cbar = fig.colorbar(im, cax=cax)
            axs[subplot_num].text(x_text_pos, 
                                  z_text_pos, 
                                  r'Viscosity, $\eta$ [Pa.s]', 
                                  rotation=90, 
                                  horizontalalignment='center', 
                                  verticalalignment='center', 
                                  zorder=9999,
                                  fontsize=SMALL_SIZE)
            imshow_box = np.array(axs[subplot_num].get_position())
            if (follow_particle):
                im = axs[subplot_num].plot(particle_x/1.0e3, particle_z/1.0e3, color="red")
                im = axs[subplot_num].scatter(particle_x[i]/1.0e3, particle_z[i]/1.0e3, color="blue", zorder=10)
            subplot_num += 1
            
        if "density" in dataset.data_vars:
            im = axs[subplot_num].imshow(data.density.T[::-1], 
                                         extent=extent, 
                                         cmap=cmap_de,
                                         norm=norm,
                                         interpolation='none')
            cax = inset_axes(axs[subplot_num], 
                             width=colorbar_width, 
                             height="100%", 
                             loc='center left', 
                             bbox_to_anchor=(colorbar_text_pos_left, 0., 0.3, 1), 
                             bbox_transform=axs[subplot_num].transAxes, 
                             borderpad=0)
            axs[subplot_num].set_aspect("equal")
            axs[subplot_num].set_ylabel("Depth [km]")
            cbar = fig.colorbar(im, cax=cax, extend='both')
            axs[subplot_num].text(x_text_pos, 
                                  z_text_pos, 
                                  r'Density, $\rho$ [kg/m$^3$]', 
                                  rotation=90, 
                                  horizontalalignment='center', 
                                  verticalalignment='center', 
                                  zorder=9999,
                                  fontsize=SMALL_SIZE)
            imshow_box = np.array(axs[subplot_num].get_position())
            subplot_num += 1

        if ("temperature" or ("velocity_x" and "velocity_x")) in dataset.data_vars:
            if "temperature" in dataset.data_vars:
                im = axs[subplot_num].imshow(data.temperature.T[::-1], 
                                             extent=extent, 
                                             vmin=t_min, vmax=t_max, 
                                             cmap="RdYlBu_r",
                                             interpolation='none')
                cax = inset_axes(axs[subplot_num], 
                                 width=colorbar_width, 
                                 height="100%", 
                                 loc='center left', 
                                 bbox_to_anchor=(colorbar_text_pos_left, 0., 0.3, 1), 
                                 bbox_transform=axs[subplot_num].transAxes, 
                                 borderpad=0)
                cbar = fig.colorbar(im, cax=cax)
                if (plot_temperature_anomaly):
                    txt_label = r'$\delta_{x}$T [$^{\circ}$C/km]'
                else:
                    txt_label = r'Temperature, T [$^{\circ}$C]'
                axs[subplot_num].text(x_text_pos, 
                                  z_text_pos, 
                                  txt_label, 
                                  rotation=90, 
                                  horizontalalignment='center', 
                                  verticalalignment='center', 
                                  zorder=9999,
                                  fontsize=SMALL_SIZE)
            if ("velocity_x" and "velocity_x") in dataset.data_vars:
                vel_aux = data[dict(x=slice(None, None, num_vectors), z=slice(None, None, num_vectors))]
                v_label = '$\overrightarrow{v} =$' + f'{np.round(float(v_key*v_max*to_mm_yr))} [mm/yr]'
                im = axs[subplot_num].quiver(vel_aux.x/1.0e3, 
                                             vel_aux.z/1.0e3 + air_thickness, 
                                             vel_aux.velocity_x.values.T, 
                                             vel_aux.velocity_z.values.T, 
                                             scale=v_scale)
                # arrow_size = 0.085 * dataset.x.max()/1.0e3
                # dd = 100
                # dx, dz = 1150 + arrow_size, 2 * dd
                # x0, z0 = dataset.x.max()/1.0e3 - dx - dd,  dataset.z.min()/1.0e3 + dd
                im = axs[subplot_num].quiverkey(im, 
                                                X=0.5,#(x0+arrow_size)/(dataset.x.max()/1.0e3),
                                                Y=-0.1,#1-(z0+dd)/(dataset.z.min()/1.0e3),
                                                U=v_key,
                                                label=v_label, 
                                                labelpos='E', 
                                                fontproperties=quiverkey_font) # fontproperties corrects an unintended behaviour where the incorrect fontsize was used during 1st run
                # im = axs[subplot_num].add_patch(FancyBboxPatch((x0, z0), dx, dz, boxstyle='round, rounding_size=25', facecolor = 'white'))
            # axs[subplot_num].set_aspect("equal")
            axs[subplot_num].set_aspect("equal")
            axs[subplot_num].set_ylabel("Depth [km]")
            imshow_box = np.array(axs[subplot_num].get_position())
            subplot_num += 1
            
        if "strain" in dataset.data_vars:
            palette = plt.cm.viridis.with_extremes(over=plt.cm.viridis.colors[-1], under=plt.cm.viridis.colors[0], bad=plt.cm.viridis.colors[0])
            im = axs[subplot_num].imshow(data.strain.T[::-1],
                                         extent=extent, 
                                         norm=LogNorm(vmin=s_min, vmax=s_max), 
                                         alpha=1.0,
                                         cmap=palette,
                                         interpolation='none')
            cax = inset_axes(axs[subplot_num], 
                             width=colorbar_width, 
                             height="100%", 
                             loc='center left', 
                             bbox_to_anchor=(colorbar_text_pos_left, 0., 0.3, 1), 
                             bbox_transform=axs[subplot_num].transAxes, 
                             borderpad=0)
            axs[subplot_num].set_aspect("equal")
            axs[subplot_num].set_ylabel("Depth [km]")
            cbar = fig.colorbar(im, cax=cax, extend='min')
            axs[subplot_num].text(x_text_pos, 
                                  z_text_pos, 
                                  r'Strain, $\epsilon$', 
                                  rotation=90, 
                                  horizontalalignment='center', 
                                  verticalalignment='center', 
                                  zorder=9999,
                                  fontsize=SMALL_SIZE)
            imshow_box = np.array(axs[subplot_num].get_position())
            subplot_num += 1
            
        if "strain_rate" in dataset.data_vars:
            if "density" in dataset.data_vars:
                im = axs[subplot_num].imshow(data.density.T[::-1], 
                                            extent=extent, 
                                            cmap=cmap_de,
                                            norm=norm,
                                            interpolation='none',
                                            alpha=0.5)
            im = axs[subplot_num].imshow(data.strain_rate.T[::-1],  
                                         extent=extent, 
                                         norm=LogNorm(vmin=e_min, vmax=e_max), 
                                         alpha=1.0,
                                         cmap=cmap_sr,
                                         interpolation='none')
            cax = inset_axes(axs[subplot_num], 
                             width=colorbar_width, 
                             height="100%", 
                             loc='center left', 
                             bbox_to_anchor=(colorbar_text_pos_left, 0., 0.3, 1), 
                             bbox_transform=axs[subplot_num].transAxes, 
                             borderpad=0)
            axs[subplot_num].set_aspect("equal")
            axs[subplot_num].set_ylabel("Depth [km]")
            cbar = fig.colorbar(im, cax=cax, extend='min')
            axs[subplot_num].text(x_text_pos, 
                                  z_text_pos, 
                                  r'Strain rate, $\dot{\epsilon}$ [s$^{-1}$]', 
                                  rotation=90, 
                                  horizontalalignment='center', 
                                  verticalalignment='center', 
                                  zorder=9999,
                                  fontsize=SMALL_SIZE)
            imshow_box = np.array(axs[subplot_num].get_position())
            subplot_num += 1

        # Fix surface plot aspect ratio and position
        if "surface" in dataset.data_vars:
            axs[surface_subplot_num].set_aspect(dataset.surface.x.max()/1.0e3/120)
            plot_box = np.array(axs[surface_subplot_num].get_position())    
            new_plot_box = Bbox([[imshow_box[0, 0], plot_box[0, 1]], [imshow_box[1, 0], plot_box[1, 1]]])
            axs[surface_subplot_num].set_position(new_plot_box)   

        axs[subplot_num-1].set_xlabel("Length [km]")
        plt.suptitle("time = {:.2f} My, step = {}".format(np.round(data.time.item(), 2), data.step.item()), ha='center', y=0.9, x=0.5)
        fig.align_ylabels(axs[:])
        plt.savefig(f"{output_path}/{model_name}_output_{str(i).zfill(6)}.png", dpi=300)
        if (plot_first==False):
            plt.close('all')

## Make video with output images

In [ ]:
line = f"ffmpeg -r 7 -i {model_path}/_output/{model_name}_output_%06d.png -c:v libx264 -vf fps=25 -pix_fmt yuv420p {model_path}/_output/{model_name}.mp4"
print(line)
if (make_video):
    !rm {model_path}/_output/{model_name}.mp4
    !ffmpeg -r 7 -i {model_path}/_output/{model_name}_output_%06d.png -c:v libx264 -vf fps=25 -pix_fmt yuv420p {model_path}/_output/{model_name}.mp4

In [ ]:
line = 'ffmpeg -ss 0 -t 10 -i {model_path}/_output/{model_name}.mp4 -vf "fps=10,scale=1080:-1:flags=lanczos,split[s0][s1];[s0]palettegen[p];[s1][p]paletteuse" -loop 0 {model_path}/_output/{model_name}.gif'
print(line)
if (make_gif):
    !rm {model_path}/_output/{model_name}.gif
    !ffmpeg -ss 0 -t 10 -i {model_path}/_output/{model_name}.mp4 -vf "fps=100,scale=1080:-1:flags=lanczos,split[s0][s1];[s0]palettegen[p];[s1][p]paletteuse" -loop 0 {model_path}/_output/{model_name}.gif